This is just a simple program to check the presence of sources in the webz.io platform through the use of its API service. The task is quite simple and functional too. It collects the sources from a google sheets file (collaborative) and checks the presence of sources and the number of entities, if present. This is again exported to the spreadsheet file.

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [ ]:
import webzio
webzio.config(token="<enter webz token here>")

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

In [ ]:
SPREADSHEET_ID = "<enter sheets id here>" 

#https://docs.google.com/spreadsheets/d/<sheets id is found here >/edit#gid=1253230244

The below code does the main task - fetching data from google sheets, checking the presence of websites in the webz platform and sending the results back to google sheets as output.

In [ ]:
def main():
    credentials = None
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_user_file("token.json",SCOPES)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            credentials = flow.run_local_server(port=0)
        with open("token.json",'w') as token:
            token.write(credentials.to_json())
    try:
        service = build('sheets','v4', credentials=credentials)
        sheets = service.spreadsheets()
        
        for row in range(3,34):
            site_address = sheets.values().get(spreadsheetId=SPREADSHEET_ID, range=f"checkwebz!A{row}").execute().get("values")[0][0]
           

            print(f"Processing {site_address}")
            output = webzio.query("filterWebContent", {"q": f"(site:{site_address})","ts": "1698481624749"})
            num_of_results = output['totalResults']
            if num_of_results>0:
                sheets.values().update(spreadsheetId=SPREADSHEET_ID, range=f"checkwebz!B{row}", valueInputOption = "USER_ENTERED", body={"values":[["Yes"]]}).execute()
                sheets.values().update(spreadsheetId=SPREADSHEET_ID, range=f"checkwebz!C{row}", valueInputOption = "USER_ENTERED", body={"values":[[num_of_results]]}).execute()
            else:
                sheets.values().update(spreadsheetId=SPREADSHEET_ID, range=f"checkwebz!B{row}", valueInputOption = "USER_ENTERED", body={"values":[["No"]]}).execute()
                sheets.values().update(spreadsheetId=SPREADSHEET_ID, range=f"checkwebz!C{row}", valueInputOption = "USER_ENTERED", body={"values":[[num_of_results]]}).execute()

   
          
    except HttpError as error:
        print(error)

In [ ]:
if __name__ == "__main__":
    main()